In this project, we are predicting trends in technology adoption and interest based on social media (Twitter) data. Specifically, the model aims to forecast the following:

1. **Volume of Discussions**: Predicting the number of tweets or social media posts related to specific technologies, gadgets, or software within a given time frame in the future (e.g., daily, weekly). This serves as an indicator of public interest and awareness levels.

2. **Sentiment Trends**: Forecasting the overall sentiment (positive, negative, neutral) associated with these technologies in the social media discourse. This could involve predicting the average sentiment score or the proportion of tweets falling into each sentiment category for upcoming days.

3. **Combination of Volume and Sentiment**: A more comprehensive approach might involve predicting both the volume of discussion and the sentiment concurrently. This dual prediction can provide a more nuanced understanding of how public interest and perception might evolve over time.

### Example Predictions
- **Before a Product Launch**: If there's an upcoming release of a new gadget, the model might predict an increase in the volume of discussion and potentially the sentiment trend leading up to and following the launch.
- **Emerging Technology Trends**: For emerging tech like augmented reality, blockchain, or new software platforms, the model could forecast how discussions (both in volume and sentiment) about these technologies will trend in the short-term future.

### Purpose of These Predictions
- **Market Insight**: These predictions can provide valuable insights for businesses, marketers, and technologists about consumer interest and sentiment trends, aiding in strategic planning and decision-making.
- **Product Strategy**: For tech companies, understanding how public interest and sentiment are likely to shift can inform product development, marketing strategies, and customer engagement plans.
- **Investment Decisions**: Investors in technology sectors might use these predictions to gauge potential market reactions to new technologies or products.

The predictions, therefore, are not just about the raw data but also about interpreting the data to extract meaningful trends and insights that can inform various strategic decisions in the technology domain.

In [1]:
# Essential imports
import pandas as pd
import numpy as np
import tweepy
import nltk
import sqlite3

## 1. Data Collection
Sources: Gather data from social media. We will be using Twitter API to search and get tweets with relevant keywords

Keywords: Identify relevant keywords for each technology (e.g., "artificial intelligence", "augmented reality", "blockchain").

In [3]:
# Twitter API keys
# Consumer Keys
api_key = '6zl6Rx7oINQIk3ZjINThyEjqG'
api_secret_key = 'TUdgoZYLxwine6O6O33pvosurWAIghU90GyP0sxwxMrOUk60dZ'

# Authentication Tokens
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAEPSjQEAAAAADjvwNAQMd146tNbpNMGZgs422m0%3D3zUK6OCejvZ60YvmYHrcxWc5zkHI5ARro9rpkxx2sDzmjM9mFs'
access_token = '2931998159-oEfo3wO1SsEkil6NJ1T3Wni7lvdciTKLIvNeUz3'
access_token_secret = 'Pu7kueCRteEwU28vzqpsCh0Y0AQ9y0wIqW8VssrZUoDDN'

# Client ID and Client Secret
client_id = 'cl9xZUpDZE9Bb01aZUdIWWQ3aFM6MTpjaQ'
client_id_secret = 'qHcKpBGB1YLgIQdRfcgMf4YCBzZpYy_OQlkf67mE_afJ1T2C3l'

# Authenticate
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

# Define a class to listen to tweets
class MyStream(tweepy.StreamingClient):
    def on_tweet(self, tweet):
        print(tweet.text)


stream = MyStream(bearer_token=bearer_token)

# Add rules (keywords) to the stream
keywords = ["artificial intelligence", "augmented reality", "blockchain"]
for keyword in keywords:
    stream.add_rules(tweepy.StreamRule(keyword))

# Start streaming
stream.filter()

Forbidden: 403 Forbidden
When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.

In [ ]:
# Storing the tweets in a database

def create_database():
    # Connect to SQLite database (it will be created if it doesn't exist)
    conn = sqlite3.connect('twitter_data.db')

    # Create a new SQLite table with columns for different tweet attributes
    conn.execute('''CREATE TABLE IF NOT EXISTS tweets
                 (id INTEGER PRIMARY KEY AUTOINCREMENT,
                  tweet_text TEXT,
                  query TEXT,
                  created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')
    
    # Commit changes and close the connection
    conn.commit()
    conn.close()

create_database()

def store_tweet(tweet_text, query):
    conn = sqlite3.connect('twitter_data.db')
    cur = conn.cursor()

    # Insert a new row of data
    cur.execute("INSERT INTO tweets (tweet_text, query) VALUES (?, ?)", (tweet_text, query))

    # Commit changes and close the connection
    conn.commit()
    conn.close()
    
# For fetching the data in the database later
def get_tweets_by_query(query):
    conn = sqlite3.connect('twitter_data.db')
    cur = conn.cursor()

    # Select tweets that match the query
    cur.execute("SELECT tweet_text FROM tweets WHERE query=?", (query,))
    all_tweets = cur.fetchall()

    conn.close()
    return all_tweets


## 2. Data Preprocessing
Cleaning: Remove irrelevant content, special characters, and URLs.
Normalization: Convert text to a standard format (e.g., lowercase, stemming).


In [ ]:
import re

def clean_tweet(tweet):
    tweet = re.sub(r'http\S+', '', tweet)  # Remove URLs
    tweet = re.sub(r'@\S+', '', tweet)  # Remove mentions
    tweet = re.sub(r'#\S+', '', tweet)  # Remove hashtags
    tweet = re.sub(r'[^A-Za-z\s]', '', tweet)  # Remove special characters and numbers
    tweet = tweet.lower()  # Convert to lowercase
    return tweet

cleaned_tweets = clean_tweet(tweets_about_product)

## 3. Sentiment Analysis
Sentiment Detection Tool: Use pre-built libraries like NLTK, TextBlob, or build a custom model using machine learning frameworks like TensorFlow or PyTorch.
Classification: Classify the sentiment of each piece of text as positive, negative, or neutral.


In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def lemmatize_tweet(tweet):
    words = tweet.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

def remove_stopwords(tweet):
    words = tweet.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

def preprocess_tweet(tweet):
    tweet = clean_tweet(tweet)
    tweet = lemmatize_tweet(tweet)
    tweet = remove_stopwords(tweet)
    return tweet

## 4. Time Series Analysis
Aggregation: Aggregate sentiment scores over time (daily, weekly).
Trends Analysis: Use time series analysis techniques to identify trends. Libraries like Pandas and statsmodels can be helpful.


## 5. Forecasting
Model Selection: Choose a forecasting model like ARIMA, SARIMA, or LSTM (for deep learning approaches).
Prediction: Use the model to predict future trends in sentiment and discussion volume.


## 6. Visualization
Tools: Use libraries like Matplotlib or Plotly to visualize trends and forecasts.
Dashboard: Consider building a dashboard using Dash or Streamlit for real-time analysis and visualization.


## 7. Continuous Improvement and Updating
Feedback Loop: Incorporate new data regularly to update the models.
Model Tuning: Continuously evaluate and tune the models for better accuracy.


## 8. Deployment
Web Application: Deploy as a web application using frameworks like Flask or Django.
APIs: Create APIs for accessing the analysis and forecasts.